###Sentiment analysis using BERT
BERT is a huge language model trained for natural languages. Transformers is the package developed by HuggingFace which consists of models that performs high performance natural language processing operations.

SMILE dataset consisting Tweets about British muesums

In [ ]:
#check if the code is running on the GPU

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
!pip install transformers

     |████████████████████████████████| 778kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 24.1MB/s 
     |████████████████████████████████| 3.0MB 30.7MB/s 
     |████████████████████████████████| 890kB 57.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=36e91db081e040eaaf51e2df867e5a7048431e4c692ba6ba510dbd0fefa256a7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random

from sklearn.model_selection import train_test_split

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertTokenizer
from transformers import BertForSequenceClassification

from transformers import AdamW, get_linear_schedule_with_warmup

from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
tweets=pd.read_csv('/content/drive/My Drive/Sentiment Analysis using BERT/smile-annotations-final.csv',
                   names=['id','text','category'])
tweets.set_index('id',inplace=True)

In [ ]:
tweets.head(10)

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy
614456889863208960,"@britishmuseum say wot, mate?",nocode
614016385442807809,Two workshops on evaluating audience engagemen...,nocode
610916556751642624,"A Forest Road, by Thomas Gainsborough 1750 Oil...",nocode


In [ ]:
tweets['category'].value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|angry               2
sad|disgust             2
sad|disgust|angry       1
Name: category, dtype: int64

In [ ]:
#removing multiple emotions and nocode

In [ ]:
tweets_modified = tweets[~tweets['category'].str.contains('\|')]
tweets_modified = tweets_modified[tweets_modified['category'] != 'nocode']

In [ ]:
# create an dictionary 
# emotions={'happy' : 0,
#           'not-relevant' : 1,
#           'angry' : 2,
#           'surprise' : 3,
#           'sad' : 4,
#           'disgust' : 5} 

In [ ]:
labels=tweets_modified.category.unique()

In [ ]:
emotions = {}
for index, label in enumerate(labels):
  emotions[label] = index

In [ ]:
emotions

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [ ]:
tweets_modified.shape

(1481, 2)

In [ ]:
tweets_modified['labels']=tweets_modified['category'].replace(emotions)

In [ ]:
tweets_modified

,text,category,labels
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0
...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0


In [ ]:
#train test split 
#stratified approach as "disgust" has very less values

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(tweets_modified.index.values,
                                               tweets_modified['labels'].values,
                                               test_size=0.15,
                                               random_state=17,
                                               stratify=tweets_modified['labels'].values)

In [ ]:
# add a coloumn to specify if the value is used for train or val

In [ ]:
tweets_modified['type']=['not-set'] * tweets_modified.shape[0]

In [ ]:
tweets_modified.head()

,text,category,labels,type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not-set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not-set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not-set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not-set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not-set


In [ ]:
tweets_modified.loc[x_train,'type']='train'
tweets_modified.loc[x_test,'type']='test'

In [ ]:
tweets_modified

,text,category,labels,type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,train
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,train
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,train
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,train
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,train
...,...,...,...,...
611258135270060033,@_TheWhitechapel @Campaignforwool @SlowTextile...,not-relevant,1,train
612214539468279808,“@britishmuseum: Thanks for ranking us #1 in @...,happy,0,train
613678555935973376,MT @AliHaggett: Looking forward to our public ...,happy,0,train


In [ ]:
tweets_modified.groupby(['category','labels','type']).count()

text
category     labels type       
angry        2      test      9
                    train    48
disgust      3      test      1
                    train     5
happy        0      test    171
                    train   966
not-relevant 1      test     32
                    train   182
sad          4      test      5
                    train    27
surprise     5      test      5
                    train    30

In [ ]:
#Preprocessing done

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
                                          do_lower_case=True)

In [ ]:
encoded_data_train=tokenizer.batch_encode_plus(tweets_modified['text'][tweets_modified['type']=='train'].values,
                                               add_special_tokens=True,
                                               return_attention_mask=True,
                                               pad_to_max_length=True,
                                               max_length=256,
                                               return_tensors='pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
encoded_data_test=tokenizer.batch_encode_plus(tweets_modified['text'][tweets_modified['type']=='test'].values,
                                               add_special_tokens=True,
                                               return_attention_mask=True,
                                               pad_to_max_length=True,
                                               max_length=256,
                                               return_tensors='pt')

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train=torch.tensor(tweets_modified['labels'][tweets_modified['type']=='train'].values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test=torch.tensor(tweets_modified['labels'][tweets_modified['type']=='test'].values)

In [ ]:
train_dataset=TensorDataset(input_ids_train,

                            attention_masks_train,
                            labels_train)

test_dataset=TensorDataset(input_ids_test,
                           attention_masks_test,
                           labels_test)

In [ ]:
len(train_dataset)

1258

In [ ]:
len(test_dataset)

223

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels=len(emotions),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
batch_size = 32
epochs = 10

In [ ]:
train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              sampler = RandomSampler(train_dataset))

test_dataloader = DataLoader(test_dataset,
                             batch_size=32,
                             sampler = RandomSampler(test_dataset))

In [ ]:
optimizer =  AdamW(model.parameters(),
                   lr = 1e-5,
                   eps = 1e-8)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer = optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = len(train_dataloader) * epochs
                                            )

In [ ]:
#performance metrics

In [ ]:
#flatting the predicted prob and then computing the f1 score
def f1_score_pred(pred, labels):
  preds_flat = np.argmax(pred, axis = 1).flatten()
  labels_flat = np.argmax(pred, axis = 1).flatten()

  return f1_score(labels_flat, preds_flat, average = 'weighted')

In [ ]:
#accuracy per class
def accuracy_per_class(preds, labels):
  label_dict_inverse = {v : k for k, v in emotions.items()}

  preds_flat = np.argmax(preds, axis = 1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat == label]
    y_true = labels_flat[labels_flat == label]
    print(f'Class : {label_dict_inverse[label]}')
    print(f'Accuracy : {len(y_preds[y_preds == label])}/{len(y_true)}\n')

In [ ]:
#training loop

In [ ]:
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()

    loss_train_total = 0

    progress_bar = tqdm(train_dataloader,
                        desc = 'Epoch {:1d}'.format(epoch),
                        leave = False,
                        disable = False)
    
    for batch in progress_bar:

      model.zero_grad()
      batch = tuple(b.to(device) for b in batch)
      inputs = {'input_ids'      : batch[0],
                'attention_mask' : batch[1],
                'labels'         : batch[2] }
      outputs = model(**inputs)
      loss = outputs[0]
      loss_train_total += loss.item()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

      optimizer.step()
      scheduler.step()

      progress_bar.set_postfix({'training_loss' : '{:.3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(),f'/content/drive/My Drive/Sentiment Analysis using BERT/BERT_ft_epoch{epoch}.model')  

    tqdm.write(f'\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(train_dataloader)
    
    tqdm.write(f'Training loss : {loss_train_avg}')

    val_loss , predictions, true_vals = evaluate(test_dataloader)
    val_f1 = f1_score_pred(predictions, true_vals)

    tqdm.write(f'Validation loss : {val_loss}')
    tqdm.write(f'F1 score (weighted) : {val_f1}')



Epoch 1
Training loss : 0.23698616605252026



Validation loss : 0.5314048528671265
F1 score (weighted) : 1.0



Epoch 2
Training loss : 0.24474123008549215



Validation loss : 0.5307539722749165
F1 score (weighted) : 1.0



Epoch 3
Training loss : 0.23514282051473856


...


Validation loss : 0.5317155995539257
F1 score (weighted) : 1.0



Epoch 9
Training loss : 0.22840791288763285



Validation loss : 0.5295699451650892
F1 score (weighted) : 1.0



Epoch 10
Training loss : 0.23889839220792056



Validation loss : 0.5300985319273812
F1 score (weighted) : 1.0



In [ ]:
_,predictions, true_values=evaluate(test_dataloader)

In [ ]:
accuracy_per_class(predictions,true_values)

Class : happy
Accuracy : 165/171

Class : not-relevant
Accuracy : 17/32

Class : angry
Accuracy : 8/9

Class : disgust
Accuracy : 0/1

Class : sad
Accuracy : 0/5

Class : surprise
Accuracy : 0/5

